import necessary libs

In [2]:
from pymatgen.core import Structure, Lattice
from pymatgen.entries.computed_entries import ComputedStructureEntry
import json 
import pandas as pd
from tqdm import tqdm
import os
from typing import Dict, Tuple
import shutil
import warnings

Next, I process each dataset from their source file into the type I require

## Alexandria

---

In [ ]:
def move_files(source_directory: str, destination_directory: str) -> None:
    """
    Moves all files from the source directory to the destination directory.
    If the destination directory does not exist, it will be created.

    :param source_directory: Path to the source directory containing the files to be moved.
    :param destination_directory: Path to the destination directory where files will be moved.
    """
    # Create the destination directory if it doesn't exist
    os.makedirs(destination_directory, exist_ok=True)

    # Walk through the source directory
    for root, dirs, files in os.walk(source_directory):
        for file in files:
            # Construct the full path of the source file
            source_file_path = os.path.join(root, file)
            # Construct the full path of the destination file
            destination_file_path = os.path.join(destination_directory, file)

            # Move the file from the source to the destination
            shutil.move(source_file_path, destination_file_path)
            print(f"Moved file: {source_file_path} -> {destination_file_path}")

source_directory = 'alexandria\\1'
destination_directory = 'alexandria\\'

move_files(source_directory, destination_directory)

In [ ]:
def open_json(path: str):
    with open(path) as file:
        data = json.load(file)
    return data['entries']

In [ ]:
def from_entry_to_cif_energy(data: Dict) -> Tuple[str, Dict]:
    """
    Converts a dictionary containing structure and energy data into a CIF string and energy-related data.

    :param data: A dictionary containing structure and energy information. Expected keys:
                 - 'structure': A dictionary with 'lattice' and 'sites' data.
                 - 'energy': The energy value of the structure.
                 - 'correction': Energy correction value.
                 - 'entry_id': Unique identifier for the entry.
                 - 'parameters': Additional parameters.
                 - 'composition': Composition of the structure.
                 - 'data': Additional data.
    :return: A tuple containing:
             - CIF string representation of the structure.
             - A dictionary with energy-related data.
    """
    # Extract lattice information and create a Lattice object
    lattice = Lattice(data['structure']['lattice']['matrix'])

    # Extract species and coordinates from the structure data
    species = []
    coords = []
    for site in data['structure']['sites']:
        for specie in site['species']:
            species.append(specie['element'])
            coords.append(site['xyz'])

    # Create a Structure object using the lattice, species, and coordinates
    structure = Structure(lattice, species, coords)

    # Create a ComputedStructureEntry object to store structure and energy data
    entry = ComputedStructureEntry(
        structure=structure,
        energy=data['energy'],
        correction=data['correction'],
        entry_id=data['entry_id'],
        parameters=data['parameters'],
        composition=data['composition'],
        data=data['data']
    )

    # Convert the structure to a CIF string
    cif_string = structure.to(fmt='cif')

    # Return the CIF string and the energy-related data
    return cif_string, entry.data

In [ ]:
def create_dataframe(files: list, path_to_callback: str):
    cifs_for_dataframe = []
    data_for_dataframe = []
    for file in tqdm(files):
        entrys = open_json(file)
        cifs_for_callback = []
        data_for_callback = []
        
        for entry in entrys:
            cif, entry_data = from_entry_to_cif_energy(entry)
            cifs_for_callback.append(cif)
            data_for_callback.append(entry_data)
        
        callback_df = pd.concat([pd.DataFrame(data_for_callback), pd.DataFrame({"cif": cifs_for_callback})], axis=1)
        callback_df.to_csv(path_to_callback + os.path.basename(file)[:-5] + ".csv")
        
        data_for_dataframe += data_for_callback
        cifs_for_dataframe += cifs_for_callback
            
    df_entry_data = pd.DataFrame(data_for_dataframe)
    df_cif = pd.DataFrame({"cif": cifs_for_dataframe})
    
    final_df = pd.concat([df_entry_data, df_cif], axis=1)
    
    return final_df            
        

In [ ]:
directory_path = 'alexandria\\jsons\\'

files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

In [ ]:
alexandria_dataframe = create_dataframe(["\\alexandria\convex_hull_pbe.json"], "alexandria\\callbacks\\")

In [ ]:
directory_path = 'alexandria\callbacks'
output_file_path = 'alexandria\\alexandria_full.csv'

csv_files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]

dataframes = []

for csv_file in csv_files:
    file_path = os.path.join(directory_path, csv_file)  
    df = pd.read_csv(file_path)
    dataframes.append(df)  

combined_df = pd.concat(dataframes, ignore_index=True)

combined_df.to_csv(output_file_path, index=False)

print(f"Все файлы объединены в: {output_file_path}")

In [ ]:
data = pd.read_csv("alexandria\\alexandria_full.csv")

In [ ]:
len(data.formula.unique())

In [ ]:
alex_pbe = pd.read_csv("alexandria/convex_hull_pbe.csv")

In [ ]:
len(alex_pbe.formula.unique())

In [ ]:
alex_pbe.columns

## Jarvis

In [ ]:
from jarvis.db.figshare import data

dft_3d = data('dft_3d')

In [ ]:
df = pd.DataFrame(dft_3d)

In [ ]:
def create_structure_from_data(data):
    """
    Создает объект Structure из предоставленных данных.

    :param data: dict, содержащий данные для создания структуры
    :return: pymatgen Structure
    """
    lattice_mat = data['lattice_mat']
    coords = data['coords']
    elements = data['elements']

    lattice = Lattice(lattice_mat)

    structure = Structure(lattice=lattice, species=elements, coords=coords)

    return structure

In [ ]:
atoms = df['atoms']
cifs_for_dataset = []
for atom in tqdm(atoms):
    cifs_for_dataset.append(create_structure_from_data(atom).to_file(fmt='cif'))

In [ ]:
df_cif = pd.DataFrame({"cif": cifs_for_dataset})
final_df = pd.concat([df, df_cif], axis=1)
final_df.to_csv('Jarvis/jarvis.csv')

In [ ]:
jarvis = pd.read_csv("Jarvis\\jarvis.csv")

In [ ]:
jarvis

In [ ]:
jarvis.columns

## AFlow

In [ ]:
import json
from urllib.request import urlopen
from urllib.error import URLError, HTTPError

API = 'http://aflow.org/API/aflux/'
MATCHBOOK = 'species(Metals)'
DIRECTIVE = '$paging(1)'
REQUEST = API + '?' + MATCHBOOK + ',' + DIRECTIVE

try:
    with urlopen(REQUEST) as response:
        data = response.read().decode('utf-8')
        if data:
            response_json = json.loads(data)
            print(response_json)
        else:
            print("Пустой ответ от сервера")
except HTTPError as e:
    print(f"HTTP ошибка: {e.code} - {e.reason}")
except URLError as e:
    print(f"Ошибка URL: {e.reason}")
except json.JSONDecodeError as e:
    print(f"Ошибка декодирования JSON: {e}")

In [ ]:
response_json

## perov_5

In [ ]:
perov = pd.read_csv("perov_5\\perov_5_file.csv")
perov

## carbon_24

In [ ]:
carbon = pd.read_csv("carbon_24\\carbon_24_file.csv")
carbon

In [ ]:
print(carbon.iloc[0].cif)

## mp_20

In [ ]:
mp = pd.read_csv("mp_20\\mp20_file.csv")
mp

In [ ]:
len(set(mp.material_id) & set(jarvis.reference))

## mpts_52

In [ ]:
mpts = pd.read_csv("mpts_52\\mpts_52.csv")
mpts

In [ ]:
len(set(mp.material_id) | set(mpts.material_id) | set(jarvis.reference))

## Merger

In [3]:
jarvis = pd.read_csv("../data/Jarvis/jarvis.csv")
mpts = pd.read_csv("../data/mpts_52/mpts_52.csv")
mp = pd.read_csv("../data/mp_20/mp_20.csv")
carbon = pd.read_csv("../data/carbon_24/carbon_24.csv")
alex_pbe = pd.read_csv("../data/alexandria/alexandria_pbe.csv")
perov = pd.read_csv("../data/perov_5/perov_5.csv")

C:\Users\user\AppData\Local\Temp\ipykernel_16240\267735639.py:1: DtypeWarning: Columns (14,17,26,30) have mixed types. Specify dtype option on import or set low_memory=False.
  jarvis = pd.read_csv("../data/Jarvis/jarvis.csv")


## Merge with all columns

### jarvis

In [5]:
jarvis.head()

,Unnamed: 0,jid,spg_number,spg_symbol,formula,formation_energy_peratom,func,optb88vdw_bandgap,atoms,slme,...,poisson,raw_files,nat,bulk_modulus_kv,shear_modulus_gv,mbj_bandgap,hse_gap,reference,search,cif
0,0,JVASP-90856,129,P4/nmm,TiCuSiAs,-0.42762,OptB88vdW,0.000,"{'lattice_mat': [[3.566933224304235, 0.0, -0.0...",na,...,na,[],8,na,na,na,na,mp-1080455,-As-Cu-Si-Ti,# generated using pymatgen\ndata_TiCuSiAs\n_sy...
1,1,JVASP-86097,221,Pm-3m,DyB6,-0.41596,OptB88vdW,0.000,"{'lattice_mat': [[4.089078911208881, 0.0, 0.0]...",na,...,na,"['OPT-LOPTICS,JVASP-86097.zip,https://ndownloa...",7,na,na,na,na,mp-568319,-B-Dy,# generated using pymatgen\ndata_DyB6\n_symmet...
2,2,JVASP-64906,119,I-4m2,Be2OsRu,0.04847,OptB88vdW,0.000,"{'lattice_mat': [[-1.833590720595598, 1.833590...",na,...,na,"['OPT-LOPTICS,JVASP-64906.zip,https://ndownloa...",4,na,na,na,na,auid-3eaf68dd483bf4f4,-Be-Os-Ru,# generated using pymatgen\ndata_Be2OsRu\n_sym...
3,3,JVASP-98225,14,P2_1/c,KBi,-0.44140,OptB88vdW,0.472,"{'lattice_mat': [[7.2963518353359165, 0.0, 0.0...",na,...,na,[],32,na,na,na,na,mp-31104,-Bi-K,# generated using pymatgen\ndata_KBi\n_symmetr...
4,4,JVASP-10,164,P-3m1,VSe2,-0.71026,OptB88vdW,0.000,"{'lattice_mat': [[1.6777483798834445, -2.90594...",na,...,0.23,"['FD-ELAST,JVASP-10.zip,https://ndownloader.fi...",3,48.79,33.05,0.0,na,mp-694,-Se-V,# generated using pymatgen\ndata_VSe2\n_symmet...


In [6]:
jarvis.rename(columns={'formation_energy_peratom' : 'formation_energy_per_atom'}, inplace=True)

In [8]:
jarvis['ref'] = 'jarvis'

In [9]:
jarvis.head(2)

,Unnamed: 0,jid,spg_number,spg_symbol,formula,formation_energy_per_atom,func,optb88vdw_bandgap,atoms,slme,...,raw_files,nat,bulk_modulus_kv,shear_modulus_gv,mbj_bandgap,hse_gap,reference,search,cif,ref
0,0,JVASP-90856,129,P4/nmm,TiCuSiAs,-0.42762,OptB88vdW,0.0,"{'lattice_mat': [[3.566933224304235, 0.0, -0.0...",na,...,[],8,na,na,na,na,mp-1080455,-As-Cu-Si-Ti,# generated using pymatgen\ndata_TiCuSiAs\n_sy...,jarvis
1,1,JVASP-86097,221,Pm-3m,DyB6,-0.41596,OptB88vdW,0.0,"{'lattice_mat': [[4.089078911208881, 0.0, 0.0]...",na,...,"['OPT-LOPTICS,JVASP-86097.zip,https://ndownloa...",7,na,na,na,na,mp-568319,-B-Dy,# generated using pymatgen\ndata_DyB6\n_symmet...,jarvis


### mpts

In [10]:
mpts.head(2)

,Unnamed: 0,material_id,theoretical,energy_above_hull,formation_energy_per_atom,year,cif
0,66,mp-66,False,0.136413,0.136413,1913,# generated using pymatgen\ndata_C\n_symmetry_...
1,22862,mp-22862,False,0.000000,-2.109608,1913,# generated using pymatgen\ndata_NaCl\n_symmet...


In [11]:
mpts['ref'] = 'mpts_52'

### mp

In [13]:
mp.head(2)

,Unnamed: 0,material_id,formation_energy_per_atom,band_gap,pretty_formula,e_above_hull,elements,cif,spacegroup.number
0,37228,mp-1221227,-1.637460,0.2133,Na3MnCoNiO6,0.043001,"['Co', 'Mn', 'Na', 'Ni', 'O']",# generated using pymatgen\ndata_Na3MnCoNiO6\n...,8
1,19480,mp-974729,-0.314759,0.0000,Nd(Al2Cu)4,0.000000,"['Al', 'Cu', 'Nd']",# generated using pymatgen\ndata_Nd(Al2Cu)4\n_...,139


In [14]:
mp['ref'] = 'mp_20'

In [22]:
mp.rename(columns={"pretty_formula": "formula"}, inplace=True)
mp.head(2)

,Unnamed: 0,material_id,formation_energy_per_atom,band_gap,formula,e_above_hull,elements,cif,spacegroup.number,ref
0,37228,mp-1221227,-1.637460,0.2133,Na3MnCoNiO6,0.043001,"['Co', 'Mn', 'Na', 'Ni', 'O']",# generated using pymatgen\ndata_Na3MnCoNiO6\n...,8,mp_20
1,19480,mp-974729,-0.314759,0.0000,Nd(Al2Cu)4,0.000000,"['Al', 'Cu', 'Nd']",# generated using pymatgen\ndata_Nd(Al2Cu)4\n_...,139,mp_20


### carbon_24

In [17]:
carbon.head(2)

,Unnamed: 0,material_id,cif,energy_per_atom
0,9251,C-130499-1826-36,# generated using pymatgen\ndata_C\n_symmetry_...,-154.311891
1,6412,C-13904-4247-31,# generated using pymatgen\ndata_C\n_symmetry_...,-154.332183


In [18]:
carbon['ref'] = 'carbon_24'

### alex_pbe

In [20]:
alex_pbe.head(2)

,Unnamed: 0,mat_id,prototype_id,location,formula,elements,spg,nsites,stress,energy_total,...,band_gap_ind,band_gap_dir,dos_ef,energy_corrected,e_above_hull,e_form,e_phase_separation,decomposition,in_nomad,cif
0,0,agm048224876,AB3C4_344_spg139,m3gnet/alignn_009/Al/Al4Ru3Ir/xxx_02a-00_agm04...,Al4IrRu3,"['Al', 'Ir', 'Ru']",139,8,"[[5.2623353, 0.0, 0.0], [0.0, 5.2623353, 0.0],...",-57.848718,...,0.0,0.0000,5.832879,-57.848717,0.0,-0.772252,-0.002118,Al4IrRu3,NaN,# generated using pymatgen\ndata_Al4IrRu3\n_sy...
1,1,agm002302240,ABCD_0_spg129,substitutions/batch-003/O/OAgTeEu/xxx_02p-00_a...,EuAgTeO,"['Eu', 'Te', 'O', 'Ag']",129,8,"[[1.169884, 0.0, 0.0], [0.0, 1.169884, 0.0], [...",-54.629588,...,0.0,0.0107,9.840907,-56.034170,0.0,-1.697882,-0.025832,EuAgTeO,NaN,# generated using pymatgen\ndata_EuAgTeO\n_sym...


In [23]:
alex_pbe['ref'] = 'alex_pbe'

### perov

In [25]:
perov.head(2)

,Unnamed: 0,material_id,cif,formula,heat_all,heat_ref,dir_gap,ind_gap
0,8989,6334,# generated using pymatgen\ndata_TlCoN2O\n_sym...,CoTlON2,2.72,2.624864,0.0,0.0
1,4450,17313,# generated using pymatgen\ndata_CrTeN3\n_symm...,CrTeN3,1.86,1.755497,0.0,0.0


In [26]:
perov['ref'] = 'perov_5'

### merging

In [27]:
genCry_full_columns = pd.concat([alex_pbe, mp, mpts, perov, carbon, jarvis], ignore_index=True, sort=False)
genCry_full_columns.head(2)

,Unnamed: 0,mat_id,prototype_id,location,formula,elements,spg,nsites,stress,energy_total,...,density,poisson,raw_files,nat,bulk_modulus_kv,shear_modulus_gv,mbj_bandgap,hse_gap,reference,search
0,0,agm048224876,AB3C4_344_spg139,m3gnet/alignn_009/Al/Al4Ru3Ir/xxx_02a-00_agm04...,Al4IrRu3,"['Al', 'Ir', 'Ru']",139.0,8.0,"[[5.2623353, 0.0, 0.0], [0.0, 5.2623353, 0.0],...",-57.848718,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,agm002302240,ABCD_0_spg129,substitutions/batch-003/O/OAgTeEu/xxx_02p-00_a...,EuAgTeO,"['Eu', 'Te', 'O', 'Ag']",129.0,8.0,"[[1.169884, 0.0, 0.0], [0.0, 1.169884, 0.0], [...",-54.629588,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
genCry_full_columns.drop_duplicates(subset="cif", inplace=True)
genCry_full_columns.shape

(315397, 95)

In [ ]:
genCry_full_columns['cif'].isna().sum()

np.int64(0)

In [35]:
genCry_full_columns.reset_index().drop("index", axis=1)
genCry_full_columns.drop("Unnamed: 0", axis=1)

,mat_id,prototype_id,location,formula,elements,spg,nsites,stress,energy_total,total_mag,...,density,poisson,raw_files,nat,bulk_modulus_kv,shear_modulus_gv,mbj_bandgap,hse_gap,reference,search
0,agm048224876,AB3C4_344_spg139,m3gnet/alignn_009/Al/Al4Ru3Ir/xxx_02a-00_agm04...,Al4IrRu3,"['Al', 'Ir', 'Ru']",139.0,8.0,"[[5.2623353, 0.0, 0.0], [0.0, 5.2623353, 0.0],...",-57.848718,0.000007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,agm002302240,ABCD_0_spg129,substitutions/batch-003/O/OAgTeEu/xxx_02p-00_a...,EuAgTeO,"['Eu', 'Te', 'O', 'Ag']",129.0,8.0,"[[1.169884, 0.0, 0.0], [0.0, 1.169884, 0.0], [...",-54.629588,12.755434,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,agm003161479,ABC_2_spg156,extra/batch-003/Ca/CaAgTl/xxx_02a-00_agm003161...,CaTlAg,"['Tl', 'Ca', 'Ag']",187.0,3.0,"[[5.075635, 0.0, 0.0], [0.0, 5.075635, 0.0], [...",-8.141593,-0.000005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,agm003330027,A2B3C3_126_spg63,cgat_comp/ternaries/A2B3C3/runs/batch-000/Ge/G...,Tm3Ge3Ir2,"['Tm', 'Ge', 'Ir']",63.0,16.0,"[[1.6191897, 0.0, 0.0], [0.0, 1.38523, 0.0], [...",-104.925494,-0.000002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,agm002198109,ABC4_5_spg63,substitutions/batch-002/Al/Al4PdEu/xxx_02p-00_...,EuAl4Pd,"['Eu', 'Al', 'Pd']",63.0,12.0,"[[3.8797123, 0.0, 0.4694527], [0.0, 1.4339213,...",-67.389274,13.648556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335425,NaN,NaN,NaN,AcRh2Pb2,NaN,139.0,NaN,NaN,NaN,NaN,...,11.194,na,[],5.0,na,na,na,na,1206834,-Ac-Pb-Rh
335426,NaN,NaN,NaN,PrTlZn,NaN,216.0,NaN,NaN,NaN,NaN,...,8.517,na,[],3.0,na,na,na,na,915022,-Pr-Tl-Zn
335427,NaN,NaN,NaN,BaIn2Bi2,NaN,139.0,NaN,NaN,NaN,NaN,...,7.460,0.67,[],5.0,30.67,-11.44,na,na,1214095,-Ba-Bi-In
335428,NaN,NaN,NaN,TmSi2Tc2,NaN,139.0,NaN,NaN,NaN,NaN,...,8.212,na,[],5.0,na,na,na,na,1206745,-Si-Tc-Tm


In [37]:
genCry_full_columns.to_csv("../data/genCry_columns.csv")

## Merge to mat_id\cif\ref

### jarvis -> jarvis_cry

In [29]:
jarvis_cry = jarvis[[ "reference", "cif"]]

In [30]:
jarvis_cry.columns = ["material_id", "cif"]

In [31]:
jarvis_cry['ref'] = 'jarvis'
jarvis_cry

C:\Users\user\AppData\Local\Temp\ipykernel_15744\940680558.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jarvis_cry['ref'] = 'jarvis'


,material_id,cif,ref
0,mp-1080455,# generated using pymatgen\ndata_TiCuSiAs\n_sy...,jarvis
1,mp-568319,# generated using pymatgen\ndata_DyB6\n_symmet...,jarvis
2,auid-3eaf68dd483bf4f4,# generated using pymatgen\ndata_Be2OsRu\n_sym...,jarvis
3,mp-31104,# generated using pymatgen\ndata_KBi\n_symmetr...,jarvis
4,mp-694,# generated using pymatgen\ndata_VSe2\n_symmet...,jarvis
...,...,...,...
75988,1206834,# generated using pymatgen\ndata_Ac(RhPb)2\n_s...,jarvis
75989,915022,# generated using pymatgen\ndata_PrTlZn\n_symm...,jarvis
75990,1214095,# generated using pymatgen\ndata_Ba(InBi)2\n_s...,jarvis
75991,1206745,# generated using pymatgen\ndata_Tm(SiTc)2\n_s...,jarvis


In Jarvis 3 type of datasets with different prefixes

- Material Project with prefix 'mp-'
- Aflow with prefix 'auid'
- Uniq Jarvis without prefix

### mpts_52 -> mpts_cry

In [32]:
mpts_cry = mpts[["material_id", "cif"]]

In [33]:
mpts_cry['ref'] = 'mpts_52'
mpts_cry

C:\Users\user\AppData\Local\Temp\ipykernel_15744\4020229141.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mpts_cry['ref'] = 'mpts_52'


,material_id,cif,ref
0,mp-66,# generated using pymatgen\ndata_C\n_symmetry_...,mpts_52
1,mp-22862,# generated using pymatgen\ndata_NaCl\n_symmet...,mpts_52
2,mp-30,# generated using pymatgen\ndata_Cu\n_symmetry...,mpts_52
3,mp-2741,# generated using pymatgen\ndata_CaF2\n_symmet...,mpts_52
4,mp-1455,# generated using pymatgen\ndata_MnS2\n_symmet...,mpts_52
...,...,...,...
40471,mp-11924,# generated using pymatgen\ndata_Rb2TaCuSe4\n_...,mpts_52
40472,mp-11775,# generated using pymatgen\ndata_RbUO3\n_symme...,mpts_52
40473,mp-31118,# generated using pymatgen\ndata_SmScO3\n_symm...,mpts_52
40474,mp-11778,# generated using pymatgen\ndata_ThMnSe3\n_sym...,mpts_52


### mp_20 -> mp_cry

In [35]:
mp_cry = mp[['material_id', "cif"]]
mp_cry['ref'] = 'mp_20'
mp_cry

C:\Users\user\AppData\Local\Temp\ipykernel_15744\3588085306.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mp_cry['ref'] = 'mp_20'


,material_id,cif,ref
0,mp-1221227,# generated using pymatgen\ndata_Na3MnCoNiO6\n...,mp_20
1,mp-974729,# generated using pymatgen\ndata_Nd(Al2Cu)4\n_...,mp_20
2,mp-1185360,# generated using pymatgen\ndata_LiMnIr2\n_sym...,mp_20
3,mp-1188861,# generated using pymatgen\ndata_LiCSN\n_symme...,mp_20
4,mp-677272,# generated using pymatgen\ndata_La2EuS4\n_sym...,mp_20
...,...,...,...
45224,mp-1023925,# generated using pymatgen\ndata_WS2\n_symmetr...,mp_20
45225,mp-1187764,# generated using pymatgen\ndata_Y2ZnPt\n_symm...,mp_20
45226,mp-1219588,# generated using pymatgen\ndata_RbMgCoF6\n_sy...,mp_20
45227,mp-3589,# generated using pymatgen\ndata_BPO4\n_symmet...,mp_20


### carbon_24 -> carbon_cry

In [36]:
carbon_cry = carbon.drop(["energy_per_atom", 'Unnamed: 0'], axis=1)

In [37]:
carbon_cry['ref'] = 'carbon_24'
carbon_cry

,material_id,cif,ref
0,C-130499-1826-36,# generated using pymatgen\ndata_C\n_symmetry_...,carbon_24
1,C-13904-4247-31,# generated using pymatgen\ndata_C\n_symmetry_...,carbon_24
2,C-92138-4782-35,# generated using pymatgen\ndata_C\n_symmetry_...,carbon_24
3,C-192672-505-73,# generated using pymatgen\ndata_C\n_symmetry_...,carbon_24
4,C-193956-5355-22,# generated using pymatgen\ndata_C\n_symmetry_...,carbon_24
...,...,...,...
10148,C-193926-6320-8,# generated using pymatgen\ndata_C\n_symmetry_...,carbon_24
10149,C-13665-1988-16,# generated using pymatgen\ndata_C\n_symmetry_...,carbon_24
10150,C-142855-9438-3,# generated using pymatgen\ndata_C\n_symmetry_...,carbon_24
10151,C-13640-2755-30,# generated using pymatgen\ndata_C\n_symmetry_...,carbon_24


### alexandria_pbe -> alex_pbe_cry

In [38]:
alex_pbe_cry = alex_pbe[["mat_id", "cif"]]

In [39]:
alex_pbe_cry.columns = ["material_id", "cif"]

In [40]:
alex_pbe_cry['ref'] = 'alex_pbe'
alex_pbe_cry

C:\Users\user\AppData\Local\Temp\ipykernel_15744\496001735.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alex_pbe_cry['ref'] = 'alex_pbe'


,material_id,cif,ref
0,agm048224876,# generated using pymatgen\ndata_Al4IrRu3\n_sy...,alex_pbe
1,agm002302240,# generated using pymatgen\ndata_EuAgTeO\n_sym...,alex_pbe
2,agm003161479,# generated using pymatgen\ndata_CaTlAg\n_symm...,alex_pbe
3,agm003330027,# generated using pymatgen\ndata_Tm3Ge3Ir2\n_s...,alex_pbe
4,agm002198109,# generated using pymatgen\ndata_EuAl4Pd\n_sym...,alex_pbe
...,...,...,...
144646,agm005618834,# generated using pymatgen\ndata_Sm3Ga2Ir7\n_s...,alex_pbe
144647,agm003208591,# generated using pymatgen\ndata_Li(Fe3Ge2)2\n...,alex_pbe
144648,agm005866520,# generated using pymatgen\ndata_Nb12Sn3W\n_sy...,alex_pbe
144649,agm003157526,# generated using pymatgen\ndata_CeP\n_symmetr...,alex_pbe


### perov_5 -> perov_cry

In [41]:
perov_cry = perov[["material_id", "cif"]]

for understand diff between Jarvis and perov_5 datasets let's add prefix 'p-' for perov srtuctures

In [42]:
perov_cry['material_id'] = 'p-' + perov_cry['material_id'].astype(str)

C:\Users\user\AppData\Local\Temp\ipykernel_15744\2236623699.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perov_cry['material_id'] = 'p-' + perov_cry['material_id'].astype(str)


In [43]:
perov_cry['ref'] = 'perov_5'
perov_cry

C:\Users\user\AppData\Local\Temp\ipykernel_15744\1754439102.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perov_cry['ref'] = 'perov_5'


,material_id,cif,ref
0,p-6334,# generated using pymatgen\ndata_TlCoN2O\n_sym...,perov_5
1,p-17313,# generated using pymatgen\ndata_CrTeN3\n_symm...,perov_5
2,p-6840,# generated using pymatgen\ndata_SbTeN2O\n_sym...,perov_5
3,p-11379,# generated using pymatgen\ndata_NiPtO2F\n_sym...,perov_5
4,p-16590,# generated using pymatgen\ndata_YGeN3\n_symme...,perov_5
...,...,...,...
18923,p-6452,# generated using pymatgen\ndata_CsNbN2O\n_sym...,perov_5
18924,p-7673,# generated using pymatgen\ndata_FePbN2O\n_sym...,perov_5
18925,p-819,# generated using pymatgen\ndata_NaGeNO2\n_sym...,perov_5
18926,p-11261,# generated using pymatgen\ndata_TeAuO2F\n_sym...,perov_5


### concat

In [44]:
genCry = pd.concat([mp_cry, mpts_cry, jarvis_cry, alex_pbe_cry, carbon_cry, perov_cry], ignore_index=True)

Jarvis, mp_20 and mpts_52 have crystalls from material project, so necessary to drop duplicates of this structures

In [45]:
genCry = genCry.drop_duplicates(subset=['material_id', 'cif'])

In [46]:
genCry = genCry.reset_index().drop("index", axis=1)
genCry

,material_id,cif,ref
0,mp-1221227,# generated using pymatgen\ndata_Na3MnCoNiO6\n...,mp_20
1,mp-974729,# generated using pymatgen\ndata_Nd(Al2Cu)4\n_...,mp_20
2,mp-1185360,# generated using pymatgen\ndata_LiMnIr2\n_sym...,mp_20
3,mp-1188861,# generated using pymatgen\ndata_LiCSN\n_symme...,mp_20
4,mp-677272,# generated using pymatgen\ndata_La2EuS4\n_sym...,mp_20
...,...,...,...
315444,p-6452,# generated using pymatgen\ndata_CsNbN2O\n_sym...,perov_5
315445,p-7673,# generated using pymatgen\ndata_FePbN2O\n_sym...,perov_5
315446,p-819,# generated using pymatgen\ndata_NaGeNO2\n_sym...,perov_5
315447,p-11261,# generated using pymatgen\ndata_TeAuO2F\n_sym...,perov_5


In [47]:
genCry.to_csv("../data/genCry.csv")

## Val

In [48]:
genCry = pd.read_csv("../data/genCry.csv")

mpts_cry have some structures with 1-2 atoms and we need to drop it & some cifs can be broken

In [56]:
from pymatgen.core.structure import Structure
from pymatgen.io.cif import CifParser
from io import StringIO
from typing import List, Optional

def filter_structures_from_dataframe(
    dataframe: pd.DataFrame,
    cif_column: str = "cif",
    material_id_column: str = "material_id",
    ref_column: str = "ref",
    num_of_atoms: int = 2,
    output_file: Optional[str] = None
) -> pd.DataFrame:
    """
    Filters structures from a DataFrame containing CIF strings based on the number of atoms.
    Returns a new DataFrame with only the structures that have more than the specified number of atoms.

    :param dataframe: Input DataFrame containing CIF strings and material IDs.
    :param cif_column: Name of the column in the DataFrame that contains the CIF strings. Default is "cif".
    :param material_id_column: Name of the column in the DataFrame that contains the material IDs. Default is "material_id".
    :param num_of_atoms: Minimum number of atoms a structure must have to be included in the result. Default is 2.
    :param ref_column: Name of the column in the DataFrame that contains reference data. Default is "ref".
    :param output_file: Path to the file where the filtered DataFrame will be saved. If None, the file is not saved.
    :return: A DataFrame containing the filtered structures with their material IDs and CIF strings.
    """
    warnings.filterwarnings("ignore", category=UserWarning)
    
    filtered_data = []

    for index, row in tqdm(dataframe.iterrows(), total=len(dataframe)):
        mat_id = row[material_id_column]
        cif_string = row[cif_column]
        ref = row[ref_column]

        try:
            # Convert the CIF string into a file-like object for parsing
            cif_file_like = StringIO(cif_string)

            # Parse the CIF string into a pymatgen Structure object
            parser = CifParser(cif_file_like)
            structure = parser.get_structures()[0]

            # Check if the structure has more than the specified number of atoms
            if len(structure) > num_of_atoms:
                filtered_data.append({
                    material_id_column: mat_id,
                    cif_column: cif_string,
                    ref_column: ref
                })
        except Exception as e:
            print(f"Error processing row {index}: {e}")

    filtered_df = pd.DataFrame(filtered_data).reset_index().drop("index", axis=1)
    
    if output_file:
        filtered_df.to_csv(output_file, index=False)
        print(f"Filtered data saved to file: {output_file}")
    
    return filtered_df

In [57]:
genCry_filtered = filter_structures_from_dataframe(genCry, output_file="../data/genCry_f.csv")

  0%|          | 0/315449 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Temp\ipykernel_15744\2600575588.py:41: FutureWarning: get_structures is deprecated; use parse_structures in pymatgen.io.cif instead.
The only difference is that primitive defaults to False in the new parse_structures method.So parse_structures(primitive=True) is equivalent to the old behavior of get_structures().
  structure = parser.get_structures()[0]
 24%|██▎       | 74315/315449 [04:56<09:28, 424.36it/s]

Error processing row 74238: Invalid CIF file with no structures!


 31%|███▏      | 98861/315449 [06:03<09:46, 369.02it/s]

Error processing row 98821: Invalid CIF file with no structures!


 32%|███▏      | 102217/315449 [06:11<08:12, 433.13it/s]

...

Error processing row 279024: Invalid CIF file with no structures!


 90%|████████▉ | 282836/315449 [18:27<02:06, 257.80it/s]

Error processing row 282787: Invalid CIF file with no structures!


100%|██████████| 315449/315449 [20:00<00:00, 262.77it/s]


Filtered data saved to file: ../data/genCry_f.csv
